# Транзакции, ACID, MVCC, WAL.

https://edu.postgrespro.ru/dba1/dba1_05_arch_mvcc.pdf

## Перечислить требования ACID к транзакционной системе;

![acid](attest_image\trans.png)

## Каким средством реализовано выполнение этих требований в PostgreSQL?

### MVCC

![mvcc](attest_image\mvcc.png)

![snap](attest_image\snap.png)

Многоверсионность (multiversion concurrency control) – один из возможных способов организации доступа к данным. Из четырех классических требований ACID к управлению транзакциями этот механизм имеет непосредственное отношение к атомарности (транзакция либо выполняется полностью, либо полностью отменяется), согласованности (транзакция сохраняет целостность данных) и изоляции(одновременно выполняющиеся транзакции не должны влиять друг на друга).

Механизм состоит в поддержке на низком уровне одновременно нескольких версий данных. Транзакции не видят этого; они работают соснимком, который из многих версий составляет согласованную на определенный момент времени картину данных. В зависимости от уровня изоляции, снимок может определяться в момент начала транзакции (уровени repeatable read, serializable) или отдельно для каждой операции (уровень read committed).

Таким образом, транзакции смотрят на данные через призму снимков и могут видеть разную (но согласованную) информацию. Разумеется, снимок не является полной физической копией всех данных: это только логическое представление, и его можно организовать по-разному. Простой способ состоит в полном ограничении одновременного доступа: и изменений, и чтений. Но при эффективной реализации – как в Постгресе и Оракле – читающая транзакция никогда не будет заблокирована другими транзакциями, читающими или изменяющими те же данные – каждая из них будет независимо работать со своей версией. Блокироваться будут только попытки изменить данные, которые уже изменены другой транзакцией, но еще не зафиксированы.

### WAL

https://edu.postgrespro.ru/dba1/dba1_06_arch_wal.pdf

![wal](attest_image\wal.png)

что когда при сбое пропадает содержимое оперативной памяти, для восстановления необходим журнал предзаписи. Размер необходимых файлов журнала и время восстановления ограничены благодаря периодически выполняемой контрольной точке.

В предыдущих статьях мы уже посмотрели на довольно большое число важных настроек, так или иначе относящихся к журналу. В этой статье (последней в этом цикле) мы рассмотрим те вопросы настройки, которые еще не обсуждались: уровни журнала и их назначение, а также надежность и производительность журналирования.

Уровни журнала:  
    - minimal
    - replica
    - logical

# Перечислить уровни изоляции транзакций. Отличие Read Committed от Repeatable Read. В каких случаях необходимо использовать Repeatable Read?

![iso](attest_image\iso.png)

**repeatable read** -  Этот уровень удобен,
например, для отчетов, состоящих из нескольких запросов.

# Сборка мусора 


![iso](attest_image\vacum.png)

# Индексы